## Variables
Variables are named locations in memory which are used to store values; values used in experssions as well as the results of expressions. All variables in PowerShell are declared and referenced with the `$` character.

In [1]:
$myVar = "This is a string variable"
$storedVar = $myVar
$myVar
$myVar = 12
$myVar
$myVar = $storedVar
$myVar

This is a string variable
12
This is a string variable


There are three kinds of variable used in PowerShell; user defined, automatic, and preference.

**User-Defined**: as the name implies, variables set by the user.  
**Automatic**: variables created by the PowerShell engine to track state, importantly automatic variables cannot be changed by the user.  
**Preference**: global user configuration, populated with default values but changeable by the user. 

In [30]:
"myVar is a user-defined variable defined in a previous code cell: " + $myVar
"The installation directory of the current PowerShell runtime is stored in the automatic variable PSHome: " + $PSHome
"The preference variable 'MaximumHistoryCount' Determines how many commands are saved in the command history for the current session:" + $MaximumHistoryCount


myVar is a user-defined variable defined in a previous code cell: This is a string variable
The installation directory of the current PowerShell runtime is stored in the automatic variable PSHome: C:\Users\alex\.nuget\packages\microsoft.dotnet-interactive\1.0.520801\tools\net8.0\any\runtimes\win\lib\net8.0
The preference variable 'MaximumHistoryCount' Determines how many commands are saved in the command history for the current session:4096


An important thing to note when interacting with string literals in variables is qoutes; double qoutes are expandable strings, meaning variables embedded in them will automatically expand when the expression is evaluated. Sub-expressions can even be embedded and evaluated when the overall expression is evaluated.

In [31]:
$i = 5
"The value of i is $i"
"i plus 5 is $($i + 5)"

The value of i is 5
i plus 5 is 10


Single quoted strings are literal however. Single quotes and double can be combined to create strings with qoutes in.

In [32]:
$j = 7
'The value of J is $j'
'"Quotes are easy" said no one ever'

The value of J is $j
"Quotes are easy" said no one ever


PowerShell also has a multistring variable format called a here string, which is delimited with the `@` symbol.

In [33]:
# the scope modifier : syntax is explained below
$user = $Env:USERNAME
$date = Get-Date
$jsonObj = @"
{ 
    "name": "$user",
    "date": "$date"
}
"@ | ConvertFrom-Json

$jsonObj.name
$jsonObj.date

alex
04/20/2024 11:52:01


## Scope
Like many programming langauges PowerShell uses the concept of _scopes_ to restrict access to variables. In a PowerShell session there are three predefined scopes, as well as a scope associated with PSDrive namespaces. 

**Global**: When the PowerShell engine starts, it creates what is known as a 'runspace' - a container for everything within the current PowerShell session (inverted, a PowerShell session is the time between a runspace starting and ending). When the runspace starts, the **Global** scope container exists, and so global variables and functions such as automatic variables are contained withint the global scope.  
**Local**: the current execution scope.  
**Script**: the scope created when a script runs, and any commands contained in the script are run within the script scope. When the script is running, the script scope is the local scope.

In [34]:
$w = "Foo"
$w
function localScopeExample {
    $w = "Bar"
    $w
}
localScopeExample

Foo
Bar


A child scope is created whenever a script or function is called (the called scope therefore becomes the parent scope). Those child scopes can go onto to create futher child scopes, leading to a hierarchy of scopes. As is usual with scope behaviour, parent scopes can access child scopes, but child scopes cannot access parent scopes (apart from the global scope).

_Scope modifiers_ as the name implies explicitly change the scope of a variable. The syntax of a scope modifier is `$scope_modifier:variable`.  
There are several scope modifiers that can be used:
- `global:` specifies the variable exists in the **global** scope.
- `local:` specifies the variable exists in the **local** scope.
- `script:` specifies the variable exists in the **script** scope - if there is an ancestor script to the one running this will be the local scope of that ancestor script, or **global** if there is no ancestor.
- `private:` not so much a scope as a modifier that makes the variable only visible in the current scope regardless of where it is defined. 

In [1]:
function hidden {
    $global:global_y = "Foo"
}
hidden
$private:private_z = "Boo"
Get-Variable global_y | Select-Object Name, Value, Visibility, Options
Get-Variable private_z | Select-Object Name, Value, Visibility, Options


Name      Value Visibility Options
----      ----- ---------- -------
global_y  Foo       Public    None
private_z Boo       Public Private

